# DL Assignment 3: Group 38

## Part 1

Q1:\
The general function for the output tensor is as follows: (source: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

$$ 
\text{out}(N_i, C_{out_j}) = \cancel{\text{bias}(C_{out_j})} + \sum_{j}{C_{in} - 1}\text{weight}(C_{out_j}, k) * \text{input}(N_i,k)
$$

This can be expressed through the following pseudocode:
```
NOT DONE!
for n in batch_size:
    for c_out in channels_out:
        # output_tensor[n, c_out] = sum
        sum = 0 
        for c_in in channels_in:
            sum += weight[c_out, c_in) * input[b, c_in]
        output[n, c_out] = sum
```

Q2:\
The output height and width for a layer $l$ can be expressed as follows:

$$
n_{H|W}^{[l]} = \frac{n_{H|W}^{[l-1]} + 2p^{[l-1]}-f^{[l-1]}}{s^{[l-1]}} + 1
$$

with $s$ representing stride, $p$ representing padding, and $f$ representing filter size.

Q3:\
A pseudocode implementation for the unfold function is as follows:
see: https://pytorch.org/docs/stable/generated/torch.nn.Unfold.html#torch.nn.Unfold
```
func unfold(input_tensor, p = padding, s = stride, k = kernel)
    # Add padding
    padded_tensor = zero_tensor[batches, channels, height+2p, width+2p]

    # Insert input tensor in zero tensor with padding
    padded_tensor[:, :, p:height+p, width+p] = input_tensor

    # Find and store patches
    patches = []
    unfold_tensor = [kernel_size]
    # Slide window accross input and store 
    for patch in padded_tensor:
        Add patch to patches

    # Iterate along patches at every index of the kernel window
    for index in unfold_tensor:
        values = []
        for patch in patches:
            for patch_value in patch[index]
            append patch_value to values
        unfold_tensor[index] = values
```

Code below:

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
import math

### Manual Unfolding

In [2]:
# Manual Conv w/ manual unfolding
class Conv2D(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size = (3,3),
                 stride = 1, padding = 1):
        super(Conv2D, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def forward(self, input_batch):
        b, c, h, w = input_batch.size()
        
        # Add padding
        tensor_pad = torch.full((b, c, h+2*self.padding, w+2*self.padding), 0.0)
        tensor_pad[:, :, self.padding:h+self.padding, self.padding:w+self.padding] = input_batch

        # Extract patches
        _, _, h_pad, w_pad = tensor_pad.size()
        x_steps = w_pad - self.kernel_size[0] / self.stride + 1
        y_steps = h_pad - self.kernel_size[1] / self.stride + 1

        # patches = torch.zeros(1,1024,3,3)
        patches = torch.zeros(b, h * w, self.kernel_size[0], self.kernel_size[1])
        for batch in range(b):
            temp_patches = torch.empty(c, self.kernel_size[0], self.kernel_size[1])
            for channel in range(c):
                for x in range (0, int(x_steps), self.stride):
                    for y in range(0, int(y_steps), self.stride):
                        patch = tensor_pad[0,0,x:x+3,y:y+3]
                        patch = torch.unsqueeze(patch, dim = 0)
                        temp_patches = torch.cat((temp_patches, patch))
            temp_patches = temp_patches[1:] # All individual patches in one batch 
            temp_patches = torch.unsqueeze(temp_patches, dim = 0)
            print(temp_patches.size())
            patches = torch.cat((patches, temp_patches), dim = 0) # Insert patches of one batch into tensor of all patches
        patches = patches[1:]

        # Fill in values in unfold tensor
        unfold = torch.zeros(b,self.kernel_size[0] * self.kernel_size[1], h * w)
        index = 0
        for batch in range(b):
            for x in range(self.kernel_size[0]):
                for y in range(self.kernel_size[1]):
                    unfold[batch][index] = torch.unsqueeze(patches, dim = 0)[0,0,:,x,y] # Make tensor of all values at an index of the oatches
                    index += 1
        
        return unfold, patches, tensor_pad

In [3]:
conv = Conv2D(1,1)
input_tensor = torch.rand(1,1,32,32)
print("Own output:\n")
out, patch, padded = conv(input_tensor)
print(out)
print("\nPyTorch unfold output:\n")
unfoldFunc = torch.nn.Unfold((3,3), dilation = 1, padding = 1, stride = 1)
print(unfoldFunc(input_tensor))

Own output:

torch.Size([1, 1024, 3, 3])
tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.9192, 0.5909, 0.6573],
         [0.0000, 0.0000, 0.0000,  ..., 0.5909, 0.6573, 0.9361],
         [0.0000, 0.0000, 0.0000,  ..., 0.6573, 0.9361, 0.0000],
         ...,
         [0.0000, 0.8208, 0.7676,  ..., 0.0000, 0.0000, 0.0000],
         [0.8208, 0.7676, 0.3601,  ..., 0.0000, 0.0000, 0.0000],
         [0.7676, 0.3601, 0.8680,  ..., 0.0000, 0.0000, 0.0000]]])

PyTorch unfold output:

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.9192, 0.5909, 0.6573],
         [0.0000, 0.0000, 0.0000,  ..., 0.5909, 0.6573, 0.9361],
         [0.0000, 0.0000, 0.0000,  ..., 0.6573, 0.9361, 0.0000],
         ...,
         [0.0000, 0.8208, 0.7676,  ..., 0.0000, 0.0000, 0.0000],
         [0.8208, 0.7676, 0.3601,  ..., 0.0000, 0.0000, 0.0000],
         [0.7676, 0.3601, 0.8680,  ..., 0.0000, 0.0000, 0.0000]]])


### Using nn.Unfold

#### Module Implementation

In [4]:
class Conv2D(nn.Module):

    def __init__(self, in_channels, out_channels, kernel, kernel_size = (3,3),
                 stride = 1, padding = 1):
        super(Conv2D, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel = kernel
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def forward(self, input_batch):

        b, c, h, w = input_batch.size()
        output_size = self.calcSize(b, h, w)

        # Folding and unfolding functions
        unfoldFunc = nn.Unfold(kernel_size = self.kernel_size,
                               padding = self.padding, stride = self.stride)
        # Don't use the prior kernel/padding/stride params when folding
        foldFunc = nn.Fold(output_size = (output_size[2], output_size[3]), kernel_size = (1, 1)) 
        
        # Unfold input batch
        unfold = unfoldFunc(input_batch)

        # Multiply input batch with weights -> (XW) and fold into output shape
        # Appropriate transformations are applied for correct output shapes
        output_unfold = unfold.transpose(1,2).matmul(self.kernel.view(self.kernel.size(0), -1).t()).transpose(1, 2)
        output = foldFunc(output_unfold)
        assert list(output.size()) == output_size, f'{list(output.size())} generated. Need {output_size}'

        return output_unfold

    # Calculate the correct output size
    def calcSize(self, b, h, w):
        size_x = (h + 2 * self.padding - self.kernel_size[0]) / self.stride + 1
        size_y = (w + 2 * self.padding - self.kernel_size[0]) / self.stride + 1

        return [b, self.out_channels, int(size_x), int(size_y)]
    

In [5]:
# Test output
input_batch = torch.randn(16, 3, 32, 32)
weights = torch.randn(3, 3, 3, 3)
conv = Conv2D(3, 3, weights)
output_batch = conv(input_batch)

#### Function Implementation

In [6]:
class Conv2DFunc(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input_batch, kernel, stride = 1, padding = 1):
        """
        Receive tensor containing input -> return tensor contaiing output.
        ctx: context object containing info for backward pass.
        Cache objects for use in backward pass using: ctx.save_for_backward
        """

        # Forward pass
        b, c, h, w = input_batch.size()
        
        # Calculate correct output dimensions
        size_x = (h + 2 * padding - kernel.size(2)) / stride + 1
        size_y = (w + 2 * padding - kernel.size(3)) / stride + 1
        output_size = [b, kernel.size(0), int(size_x), int(size_y)]

        # Folding and unfolding functions
        unfoldFunc = nn.Unfold(kernel_size = (kernel.size(2), kernel.size(3)),
                               padding = padding, stride = stride)
        # Don't use the prior kernel/padding/stride params when folding
        foldFunc = nn.Fold(output_size = (output_size[2], output_size[3]), kernel_size = (1, 1))

        # Unfold -> tensor ops -> fold
        U = unfoldFunc(input_batch)
        UW = U.transpose(1,2).matmul(kernel.view(kernel.size(0), -1).t()).transpose(1, 2)
        Y = foldFunc(UW)
        assert list(Y.size()) == output_size, f'{list(Y.size())} generated. Need {output_size}'

        # Store for backward pass
        W = kernel.view(kernel.size(0), -1) # Format to follow computation graph shape
        ctx.save_for_backward(input_batch, kernel, U, W)
        ctx.stride = stride
        ctx.padding = padding
        
        return Y

    @staticmethod
    def backward(ctx, grad_output):
        """
        Generate tensor containing gradient of loss w.r.t. output.
        Compute gradient of loss w.r.t. input
        """
            
        # Retrieve stored objects and dimensions
        input_batch, kernel, U, W = ctx.saved_tensors
        b_in, c_in, h_in, w_in = input_batch.size()
        b_out, c_out, h_out, w_out = grad_output.size()
        stride = ctx.stride
        padding = ctx.padding

        # Backward pass
        # Reshape Y^nabla to pre-folded shape (Y'^nabla)
        Y_nabla = grad_output.view(b_out, c_out, (h_out * w_out)).transpose(1,2)

        # W^nabla = U x Y^nabla
        W_nabla = U.matmul(Y_nabla)
        # U^nabla = Y^nabla x W.T
        U_nabla = Y_nabla.matmul(W.view(W.size(0), -1)).transpose(1,2)
        assert U_nabla.size() == U.size(), f'U_nabla: {U_nabla.size()} generated. Need {U.size()}'

        # Fold U^nabla into X^nabla
        foldFunc = nn.Fold(output_size=(32, 32), kernel_size=(3, 3), dilation=1, 
                           padding=(padding, padding), stride=stride)
        X_nabla = foldFunc(U_nabla)
        assert X_nabla.size() == input_batch.size(), f'X_nabla: {X_nabla.size()} generated. Need {input_batch.size()}'
        
        return X_nabla, W_nabla

In [7]:
# Test output
input_batch = torch.randn(16, 3, 32, 32, requires_grad = True)
kernel = torch.randn(3, 3, 3, 3)
output_batch = Conv2DFunc.apply(input_batch, kernel)

In [8]:
# Example loss function
loss_fn = torch.sum
loss = loss_fn(output_batch)
print(loss.sum())

tensor(-588.8141, grad_fn=<SumBackward0>)


In [9]:
loss.backward()